In [1]:
# 04_Solution_Reuse.ipynb

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import re
from collections import Counter
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# --- Konfigurasi Path (sesuaikan dengan struktur proyek Anda) ---
BASE_DRIVE_PATH = "/content/drive/MyDrive/Semester 6/PK/UAS" # Sesuaikan jika berbeda
PATH_PROCESSED_DATA = os.path.join(BASE_DRIVE_PATH, "data/processed")
PATH_EVAL_DATA = os.path.join(BASE_DRIVE_PATH, "data/eval")
PATH_MODELS_CACHE = os.path.join(BASE_DRIVE_PATH, "models_cache") # Opsional
PATH_RESULTS = os.path.join(BASE_DRIVE_PATH, "data/results")

# Membuat direktori jika belum ada
os.makedirs(PATH_RESULTS, exist_ok=True)

# File input dari tahap-tahap sebelumnya
CASES_REPRESENTED_CSV = os.path.join(PATH_PROCESSED_DATA, "cases_represented.csv")
CASE_EMBEDDINGS_FILE = os.path.join(PATH_PROCESSED_DATA, "case_embeddings_bert.npy")
CASE_IDS_FILE = os.path.join(PATH_PROCESSED_DATA, "case_ids_bert.json")
QUERIES_JSON_FILE = os.path.join(PATH_EVAL_DATA, "queries.json")

# File output untuk tahap ini
PREDICTIONS_CSV_FILE = os.path.join(PATH_RESULTS, "predictions.csv")

In [4]:
# --- 1. Muat Data, Model, dan Embeddings ---
print("Memuat data, model, dan embeddings dari tahap sebelumnya...")

# Muat data kasus yang sudah direpresentasikan
try:
    df_cases = pd.read_csv(CASES_REPRESENTED_CSV)
    # Set case_id sebagai index untuk pencarian cepat (lookup)
    df_cases.set_index('case_id', inplace=True)
except FileNotFoundError:
    print(f"Error: File {CASES_REPRESENTED_CSV} tidak ditemukan. Pastikan Tahap 2 sudah dijalankan.")
    exit()

# Muat embeddings dan case_ids
try:
    case_embeddings = np.load(CASE_EMBEDDINGS_FILE)
    with open(CASE_IDS_FILE, 'r') as f:
        case_ids_for_embeddings = json.load(f)
except FileNotFoundError:
    print(f"Error: File embeddings ({CASE_EMBEDDINGS_FILE}) atau case_ids ({CASE_IDS_FILE}) tidak ditemukan. Pastikan Tahap 3 sudah dijalankan.")
    exit()

# Muat Model dan Tokenizer IndoBERT
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_NAME = 'indobenchmark/indobert-base-p1'
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=PATH_MODELS_CACHE)
    model = AutoModel.from_pretrained(MODEL_NAME, cache_dir=PATH_MODELS_CACHE)
    model.to(DEVICE)
    model.eval()
    print("Model dan tokenizer berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat model: {e}")
    exit()

Memuat data, model, dan embeddings dari tahap sebelumnya...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model dan tokenizer berhasil dimuat.


In [5]:
# --- 2. Definisikan Ulang Fungsi dari Tahap 3 ---
# Kita butuh fungsi get_bert_embedding dan retrieve_cases_bert

def get_bert_embedding(text, model, tokenizer, device, max_length=512):
    """Menghasilkan embedding untuk teks menggunakan model BERT."""
    if not isinstance(text, str) or text.strip() == "":
        return torch.zeros(model.config.hidden_size).cpu().numpy()
    encoded_input = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_attention_mask=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    with torch.no_grad():
        outputs = model(**encoded_input)
        last_hidden_states = outputs.last_hidden_state
    attention_mask = encoded_input['attention_mask']
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    mean_pooled = sum_embeddings / sum_mask
    return mean_pooled.cpu().numpy().flatten()

def retrieve_cases_bert(query_text, k=5):
    """Fungsi retrieval dari Tahap 3."""
    if not query_text.strip(): return [], []
    query_embedding = get_bert_embedding(query_text, model, tokenizer, DEVICE).reshape(1, -1)
    similarities = cosine_similarity(query_embedding, case_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    top_k_case_ids = [case_ids_for_embeddings[i] for i in top_k_indices]
    top_k_scores = [similarities[i] for i in top_k_indices]
    return top_k_case_ids, top_k_scores

In [6]:
# --- 3. Implementasi Algoritma Prediksi / Solution Reuse ---

def classify_amar_outcome(amar_text):
    """
    Mengklasifikasikan teks amar putusan ke dalam kategori hasil utama.
    Ini adalah pendekatan yang lebih robust daripada membandingkan teks amar secara langsung.
    Contoh untuk kasus perceraian. Anda mungkin perlu menyesuaikan pola regex ini.
    """
    if not isinstance(amar_text, str):
        return "TIDAK DIKETAHUI"

    amar_text_lower = amar_text.lower()

    # Pola untuk mengabulkan gugatan
    if re.search(r"mengabulkan\s+gugatan\s+penggugat", amar_text_lower):
        return "MENGABULKAN GUGATAN"

    # Pola untuk menolak gugatan
    if re.search(r"menolak\s+gugatan\s+penggugat", amar_text_lower):
        return "MENOLAK GUGATAN"

    # Pola untuk menyatakan gugatan tidak dapat diterima (Niet Ontvankelijke Verklaard / NO)
    if re.search(r"menyatakan\s+gugatan\s+(?:penggugat|para penggugat)\s+tidak\s+dapat\s+diterima", amar_text_lower):
        return "GUGATAN TIDAK DAPAT DITERIMA (NO)"

    # Jika tidak ada yang cocok, kembalikan kategori default
    return "LAIN-LAIN"


def predict_outcome(query_text, k=5, method='majority_vote'):
    """
    Memprediksi 'solusi' untuk query baru berdasarkan kasus-kasus serupa.
    Solusi di sini adalah kategori amar putusan.

    Args:
        query_text (str): Teks kasus baru / query.
        k (int): Jumlah kasus serupa yang akan dipertimbangkan.
        method (str): 'majority_vote' atau 'weighted_similarity'.

    Returns:
        tuple: (predicted_solution_category, top_k_case_ids)
    """
    # 1. Temukan top-k kasus serupa
    top_k_ids, top_k_scores = retrieve_cases_bert(query_text, k)

    if not top_k_ids:
        return "Tidak ada kasus serupa yang ditemukan", []

    # 2. Ekstrak dan klasifikasikan 'solusi' (amar putusan) dari top-k kasus
    solutions = []
    for case_id in top_k_ids:
        try:
            amar = df_cases.loc[case_id, 'amar_putusan']
            amar_category = classify_amar_outcome(amar)
            solutions.append(amar_category)
        except KeyError:
            print(f"Peringatan: case_id {case_id} tidak ditemukan di DataFrame. Dilewati.")
            solutions.append(None) # Atau bisa diabaikan

    if all(s is None for s in solutions):
        return "Tidak ada solusi yang bisa diekstrak", top_k_ids

    # 3. Terapkan algoritma prediksi
    predicted_solution = "Tidak dapat diprediksi"
    if method == 'majority_vote':
        # Hitung frekuensi setiap kategori solusi dan ambil yang paling umum
        vote_counts = Counter(s for s in solutions if s is not None)
        if vote_counts:
            predicted_solution = vote_counts.most_common(1)[0][0]

    elif method == 'weighted_similarity':
        # Bobot setiap suara dengan skor kemiripannya
        weighted_scores = {}
        for i, category in enumerate(solutions):
            if category is not None:
                if category not in weighted_scores:
                    weighted_scores[category] = 0
                weighted_scores[category] += top_k_scores[i] # Tambahkan bobot skor

        if weighted_scores:
            # Pilih kategori dengan total skor bobot tertinggi
            predicted_solution = max(weighted_scores, key=weighted_scores.get)

    else:
        raise ValueError("Metode tidak valid. Pilih 'majority_vote' atau 'weighted_similarity'.")

    return predicted_solution, top_k_ids

In [7]:
# --- 4. Demo Manual & Penyimpanan Hasil ---
print("\n--- Demo dan Penyimpanan Hasil Prediksi ---")

# Muat queries dari Tahap 3
try:
    with open(QUERIES_JSON_FILE, 'r', encoding='utf-8') as f:
        test_queries = json.load(f)
except FileNotFoundError:
    print(f"Error: File queries.json tidak ditemukan di {QUERIES_JSON_FILE}.")
    print("Membuat contoh query manual untuk demo.")
    test_queries = [{
        "query_id": "Q_DEMO_01",
        "query_text": "Seorang istri ditinggal pergi suaminya selama 6 tahun berturut-turut tanpa kabar dan tidak memberikan nafkah lahir batin."
    }]

prediction_results = []
if test_queries:
    for query_data in test_queries:
        query_id = query_data['query_id']
        query_text = query_data['query_text']

        print(f"\nMemproses Query ID: {query_id}")
        print(f"Query Text: \"{query_text[:150]}...\"")

        # Lakukan prediksi (kita gunakan weighted_similarity sebagai contoh)
        predicted_solution, top_5_ids = predict_outcome(query_text, k=5, method='weighted_similarity')

        print(f"-> Top 5 Similar Case IDs: {top_5_ids}")
        print(f"-> Predicted Outcome/Solution: '{predicted_solution}'")

        # Simpan hasil untuk CSV
        prediction_results.append({
            "query_id": query_id,
            "predicted_solution": predicted_solution,
            "top_5_case_ids": json.dumps(top_5_ids) # Simpan list sebagai string JSON
        })
else:
    print("Tidak ada query untuk diuji.")

# Simpan hasil prediksi ke file CSV sesuai instruksi PDF
if prediction_results:
    df_predictions = pd.DataFrame(prediction_results)
    try:
        df_predictions.to_csv(PREDICTIONS_CSV_FILE, index=False, encoding='utf-8-sig')
        print(f"\n Hasil prediksi berhasil disimpan ke: {PREDICTIONS_CSV_FILE}")
        print("Cuplikan hasil prediksi:")
        print(df_predictions)
    except Exception as e:
        print(f"Error saat menyimpan file prediksi CSV: {e}")

print("\n--- Tahap 4 Selesai ---")


--- Demo dan Penyimpanan Hasil Prediksi ---

Memproses Query ID: Q001
Query Text: "Suami meninggalkan istri dan anak lebih dari dua tahun tanpa nafkah dan kabar berita...."
-> Top 5 Similar Case IDs: [30, 2, 29, 7, 23]
-> Predicted Outcome/Solution: 'LAIN-LAIN'

Memproses Query ID: Q002
Query Text: "Terjadi perselisihan dan pertengkaran terus menerus dalam rumah tangga sehingga tidak ada harapan rukun kembali...."
-> Top 5 Similar Case IDs: [2, 30, 22, 23, 12]
-> Predicted Outcome/Solution: 'LAIN-LAIN'

Memproses Query ID: Q003
Query Text: "Salah satu pihak melakukan penganiayaan...."
-> Top 5 Similar Case IDs: [30, 7, 2, 22, 10]
-> Predicted Outcome/Solution: 'LAIN-LAIN'

 Hasil prediksi berhasil disimpan ke: /content/drive/MyDrive/Semester 6/PK/UAS/data/results/predictions.csv
Cuplikan hasil prediksi:
  query_id predicted_solution       top_5_case_ids
0     Q001          LAIN-LAIN   [30, 2, 29, 7, 23]
1     Q002          LAIN-LAIN  [2, 30, 22, 23, 12]
2     Q003          LAIN-LAIN  